In [1]:
## conda environment Weather_Prediction

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import geopandas as gpd
import xarray as xr
import regionmask

In [2]:
data = xr.open_dataset("data.nc") ## one month average of the variables
data

<xarray.Dataset> Size: 7MB
Dimensions:        (valid_time: 12, latitude: 35, longitude: 81)
Coordinates:
  * valid_time     (valid_time) datetime64[ns] 96B 2010-01-01 ... 2010-12-01
  * latitude       (latitude) float64 280B 49.5 48.75 48.0 ... 25.5 24.75 24.0
  * longitude      (longitude) float64 648B -126.0 -125.2 ... -66.75 -66.0
Data variables: (12/48)
    d2m            (valid_time, latitude, longitude) float32 136kB ...
    t2m            (valid_time, latitude, longitude) float32 136kB ...
    bcaod550       (valid_time, latitude, longitude) float32 136kB ...
    chnk           (valid_time, latitude, longitude) float32 136kB ...
    duaod550       (valid_time, latitude, longitude) float32 136kB ...
    lai_hv         (valid_time, latitude, longitude) float32 136kB ...
    ...             ...
    aermssomhphob  (valid_time, latitude, longitude) float32 136kB ...
    aermsssss      (valid_time, latitude, longitude) float32 136kB ...
    aermssssm      (valid_time, latitude, longitude) float32 136kB ...
    aermssssl      (valid_time, latitude, longitude) float32 136kB ...
    aermsssu       (valid_time, latitude, longitude) float32 136kB ...
    aermssso2      (valid_time, latitude, longitude) float32 136kB ...
Attributes:
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2024-10-10T00:09 GRIB to CDM+CF via cfgrib-0.9.1...

In [3]:
## resample the data to 1 year average

year_avg = data.resample(valid_time='1YE').mean(dim='valid_time')
year_avg

<xarray.Dataset> Size: 545kB
Dimensions:        (valid_time: 1, latitude: 35, longitude: 81)
Coordinates:
  * latitude       (latitude) float64 280B 49.5 48.75 48.0 ... 25.5 24.75 24.0
  * longitude      (longitude) float64 648B -126.0 -125.2 ... -66.75 -66.0
  * valid_time     (valid_time) datetime64[ns] 8B 2010-12-31
Data variables: (12/48)
    d2m            (valid_time, latitude, longitude) float32 11kB 278.3 ... 2...
    t2m            (valid_time, latitude, longitude) float32 11kB 281.6 ... 2...
    bcaod550       (valid_time, latitude, longitude) float32 11kB 0.006512 .....
    chnk           (valid_time, latitude, longitude) float32 11kB 0.01415 ......
    duaod550       (valid_time, latitude, longitude) float32 11kB 0.001253 .....
    lai_hv         (valid_time, latitude, longitude) float32 11kB 3.752 ... 0.0
    ...             ...
    aermssomhphob  (valid_time, latitude, longitude) float32 11kB 9.168e-06 ....
    aermsssss      (valid_time, latitude, longitude) float32 11kB 6.539e-07 ....
    aermssssm      (valid_time, latitude, longitude) float32 11kB 4.983e-05 ....
    aermssssl      (valid_time, latitude, longitude) float32 11kB 1.505e-05 ....
    aermsssu       (valid_time, latitude, longitude) float32 11kB 2.849e-06 ....
    aermssso2      (valid_time, latitude, longitude) float32 11kB 6.057e-07 ....
Attributes:
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2024-10-10T00:09 GRIB to CDM+CF via cfgrib-0.9.1...

In [4]:
## delete variables that have NaN in them

vars_with_nan = [var for var in year_avg if year_avg[var].isnull().any()]
year_avg= year_avg.drop_vars(vars_with_nan)
year_avg

<xarray.Dataset> Size: 523kB
Dimensions:        (valid_time: 1, latitude: 35, longitude: 81)
Coordinates:
  * latitude       (latitude) float64 280B 49.5 48.75 48.0 ... 25.5 24.75 24.0
  * longitude      (longitude) float64 648B -126.0 -125.2 ... -66.75 -66.0
  * valid_time     (valid_time) datetime64[ns] 8B 2010-12-31
Data variables: (12/46)
    d2m            (valid_time, latitude, longitude) float32 11kB 278.3 ... 2...
    t2m            (valid_time, latitude, longitude) float32 11kB 281.6 ... 2...
    bcaod550       (valid_time, latitude, longitude) float32 11kB 0.006512 .....
    chnk           (valid_time, latitude, longitude) float32 11kB 0.01415 ......
    duaod550       (valid_time, latitude, longitude) float32 11kB 0.001253 .....
    lai_hv         (valid_time, latitude, longitude) float32 11kB 3.752 ... 0.0
    ...             ...
    aermssomhphob  (valid_time, latitude, longitude) float32 11kB 9.168e-06 ....
    aermsssss      (valid_time, latitude, longitude) float32 11kB 6.539e-07 ....
    aermssssm      (valid_time, latitude, longitude) float32 11kB 4.983e-05 ....
    aermssssl      (valid_time, latitude, longitude) float32 11kB 1.505e-05 ....
    aermsssu       (valid_time, latitude, longitude) float32 11kB 2.849e-06 ....
    aermssso2      (valid_time, latitude, longitude) float32 11kB 6.057e-07 ....
Attributes:
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2024-10-10T00:09 GRIB to CDM+CF via cfgrib-0.9.1...

In [5]:
## note that even though the time dimesion, essentially shows 2010-12-31,
## what it actually means is yearly average
year_avg.to_dataframe()

d2m         t2m  bcaod550      chnk  \
valid_time latitude longitude                                               
2010-12-31 49.5     -126.00    278.348907  281.556824  0.006512  0.014148   
                    -125.25    279.088837  282.437836  0.006916  0.013934   
                    -124.50    279.470001  282.923370  0.007213  0.013523   
                    -123.75    279.041290  282.562347  0.007478  0.013581   
                    -123.00    277.851837  281.525085  0.008017  0.014813   
...                                   ...         ...       ...       ...   
           24.0     -69.00     293.523407  297.542999  0.004872  0.012293   
                    -68.25     293.495728  297.488953  0.004856  0.012303   
                    -67.50     293.465454  297.455261  0.004833  0.012302   
                    -66.75     293.443970  297.427094  0.004824  0.012318   
                    -66.00     293.438751  297.397156  0.004825  0.012332   

                               duaod550    lai_hv    lai_lv            msl  \
valid_time latitude longitude                                                
2010-12-31 49.5     -126.00    0.001253  3.752065  2.928701  101390.726562   
                    -125.25    0.001246  4.228424  1.811178  101402.250000   
                    -124.50    0.001238  2.194234  0.692861  101427.570312   
                    -123.75    0.001204  0.905955  0.624898  101459.429688   
                    -123.00    0.001140  3.628265  2.724324  101485.656250   
...                                 ...       ...       ...            ...   
           24.0     -69.00     0.024328  0.000000  0.000000  101571.789062   
                    -68.25     0.024477  0.000000  0.000000  101578.000000   
                    -67.50     0.024507  0.000000  0.000000  101583.164062   
                    -66.75     0.024585  0.000000  0.000000  101590.195312   
                    -66.00     0.024893  0.000000  0.000000  101595.625000   

                               omaod550         pm2p5  ...     aermssdul  \
valid_time latitude longitude                          ...                 
2010-12-31 49.5     -126.00    0.055563  5.288607e-09  ...  5.697568e-07   
                    -125.25    0.060980  6.618710e-09  ...  5.552510e-07   
                    -124.50    0.063776  7.792362e-09  ...  5.443322e-07   
                    -123.75    0.067136  1.080445e-08  ...  5.231616e-07   
                    -123.00    0.076543  1.931933e-08  ...  4.871432e-07   
...                                 ...           ...  ...           ...   
           24.0     -69.00     0.046011  9.519893e-09  ...  1.086019e-05   
                    -68.25     0.045661  9.399162e-09  ...  1.102058e-05   
                    -67.50     0.045540  9.288230e-09  ...  1.103588e-05   
                    -66.75     0.045691  9.213097e-09  ...  1.106749e-05   
                    -66.00     0.045851  9.209978e-09  ...  1.116341e-05   

                               aermssbchphil  aermssomhphil  aermssbchphob  \
valid_time latitude longitude                                                
2010-12-31 49.5     -126.00     1.359443e-07   2.107467e-06   5.559153e-07   
                    -125.25     1.506114e-07   2.379295e-06   5.843853e-07   
                    -124.50     1.616488e-07   2.564180e-06   6.047914e-07   
                    -123.75     1.798799e-07   2.806439e-06   6.146706e-07   
                    -123.00     2.252995e-07   3.389544e-06   6.264335e-07   
...                                      ...            ...            ...   
           24.0     -69.00      3.103107e-08   5.009322e-07   4.866120e-07   
                    -68.25      3.011268e-08   4.867323e-07   4.857583e-07   
                    -67.50      2.886006e-08   4.733052e-07   4.846137e-07   
                    -66.75      2.775320e-08   4.601205e-07   4.847639e-07   
                    -66.00      2.699588e-08   4.502798e-07   4.856791e-07   

                 

In [7]:
## Lets load the county shapefile

SHAPE_PATH = os.path.join("County_shapefile",'gz_2010_us_050_00_500k.shp')
county_gdf = gpd.read_file(SHAPE_PATH)

county_gdf.head(12)

,GEO_ID,STATE,COUNTY,NAME,LSAD,CENSUSAREA,geometry
0,0500000US01029,01,029,Cleburne,County,560.100,"POLYGON ((-85.38872 33.91304, -85.38088 33.873..."
1,0500000US01031,01,031,Coffee,County,678.972,"POLYGON ((-86.03044 31.61894, -86.00408 31.619..."
2,0500000US01037,01,037,Coosa,County,650.926,"POLYGON ((-86.00928 33.10164, -86.00917 33.090..."
3,0500000US01039,01,039,Covington,County,1030.456,"POLYGON ((-86.34851 30.99434, -86.35023 30.994..."
4,0500000US01041,01,041,Crenshaw,County,608.840,"POLYGON ((-86.14699 31.68045, -86.14711 31.663..."
5,0500000US01045,01,045,Dale,County,561.150,"POLYGON ((-85.79043 31.32027, -85.79033 31.323..."
6,0500000US01049,01,049,DeKalb,County,777.093,"POLYGON ((-85.57593 34.82373, -85.56142 34.750..."
7,0500000US01053,01,053,Escambia,County,945.080,"POLYGON ((-87.16308 30.99904, -87.16408 30.999..."
8,0500000US01057,01,057,Fayette,County,627.660,"POLYGON ((-87.63593 33.87874, -87.63604 33.872..."
9,0500000US01061,01,061,Geneva,County,574.408,"POLYGON ((-85.77267 30.99462, -85.77966 30.994..."


In [16]:
county_gdf.dtypes

GEO_ID                   object
STATE                    object
COUNTY                   object
NAME                     object
LSAD                     object
CENSUSAREA              float64
geometry               geometry
polygon_coordinates      object
dtype: object

In [72]:
column=county_gdf['polygon_coordinates']
column

0       [(-85.3887171312565, 33.9130442466707), (-85.3...
1       [(-86.030441, 31.618938999999997), (-86.004084...
2       [(-86.00927999999999, 33.101638), (-86.00917, ...
3       [(-86.3485063167533, 30.994339155609), (-86.35...
4       [(-86.146992, 31.680455), (-86.147114, 31.663)...
                              ...                        
3216    [(-66.90748099999999, 18.25314), (-66.907387, ...
3217    [(-66.3796774170153, 17.9439848012997), (-66.3...
3218    [[(-66.1395715759286, 18.462316555407398), (-6...
3219    [(-66.029167, 18.375905), (-66.028279, 18.3768...
3220    [(-66.8522884755987, 17.9550037242825), (-66.8...
Name: polygon_coordinates, Length: 3221, dtype: object

In [73]:
one_county=column.iloc[0]

In [74]:
one_county

[(-85.3887171312565, 33.9130442466707),
 (-85.38088499793109, 33.8735079039461),
 (-85.3794552619804, 33.8662906454797),
 (-85.377426, 33.856047),
 (-85.3764027281585, 33.8506557456616),
 (-85.36459509570929, 33.788445541993795),
 (-85.36184399999999, 33.773951),
 (-85.360491, 33.767958),
 (-85.357402, 33.750104),
 (-85.35525221574429, 33.739245277912495),
 (-85.3440544299748, 33.6826844191468),
 (-85.3427217288762, 33.675952845587),
 (-85.3237922987933, 33.5803388694423),
 (-85.3153400895111, 33.537646124170294),
 (-85.314994, 33.535897999999996),
 (-85.3148434877199, 33.5349512560896),
 (-85.31409099999999, 33.530218),
 (-85.313999, 33.529807),
 (-85.304439, 33.482884),
 (-85.308211, 33.481578999999996),
 (-85.30924999999999, 33.483137),
 (-85.314852, 33.487603),
 (-85.316028, 33.488267),
 (-85.320893, 33.488358999999996),
 (-85.324856, 33.489160999999996),
 (-85.33106099999999, 33.491014),
 (-85.33828, 33.4947),
 (-85.34254399999999, 33.495961),
 (-85.344923, 33.497608),
 (-85.34670

In [75]:
one_county

[(-85.3887171312565, 33.9130442466707),
 (-85.38088499793109, 33.8735079039461),
 (-85.3794552619804, 33.8662906454797),
 (-85.377426, 33.856047),
 (-85.3764027281585, 33.8506557456616),
 (-85.36459509570929, 33.788445541993795),
 (-85.36184399999999, 33.773951),
 (-85.360491, 33.767958),
 (-85.357402, 33.750104),
 (-85.35525221574429, 33.739245277912495),
 (-85.3440544299748, 33.6826844191468),
 (-85.3427217288762, 33.675952845587),
 (-85.3237922987933, 33.5803388694423),
 (-85.3153400895111, 33.537646124170294),
 (-85.314994, 33.535897999999996),
 (-85.3148434877199, 33.5349512560896),
 (-85.31409099999999, 33.530218),
 (-85.313999, 33.529807),
 (-85.304439, 33.482884),
 (-85.308211, 33.481578999999996),
 (-85.30924999999999, 33.483137),
 (-85.314852, 33.487603),
 (-85.316028, 33.488267),
 (-85.320893, 33.488358999999996),
 (-85.324856, 33.489160999999996),
 (-85.33106099999999, 33.491014),
 (-85.33828, 33.4947),
 (-85.34254399999999, 33.495961),
 (-85.344923, 33.497608),
 (-85.34670

In [25]:
one_county[0][1]

33.9130442466707

In [27]:
cc=county_gdf.geometry
cc

0       POLYGON ((-85.38872 33.91304, -85.38088 33.873...
1       POLYGON ((-86.03044 31.61894, -86.00408 31.619...
2       POLYGON ((-86.00928 33.10164, -86.00917 33.090...
3       POLYGON ((-86.34851 30.99434, -86.35023 30.994...
4       POLYGON ((-86.14699 31.68045, -86.14711 31.663...
                              ...                        
3216    POLYGON ((-66.90748 18.25314, -66.90739 18.253...
3217    POLYGON ((-66.37968 17.94398, -66.38029 17.943...
3218    MULTIPOLYGON (((-66.13957 18.46232, -66.13956 ...
3219    POLYGON ((-66.02917 18.37590, -66.02828 18.376...
3220    POLYGON ((-66.85229 17.95500, -66.85280 17.955...
Name: geometry, Length: 3221, dtype: geometry

In [30]:
kk=county_gdf.geometry.exterior
kk

0       LINEARRING (-85.38872 33.91304, -85.38088 33.8...
1       LINEARRING (-86.03044 31.61894, -86.00408 31.6...
2       LINEARRING (-86.00928 33.10164, -86.00917 33.0...
3       LINEARRING (-86.34851 30.99434, -86.35023 30.9...
4       LINEARRING (-86.14699 31.68045, -86.14711 31.6...
                              ...                        
3216    LINEARRING (-66.90748 18.25314, -66.90739 18.2...
3217    LINEARRING (-66.37968 17.94398, -66.38029 17.9...
3218                                                 None
3219    LINEARRING (-66.02917 18.37590, -66.02828 18.3...
3220    LINEARRING (-66.85229 17.95500, -66.85280 17.9...
Length: 3221, dtype: geometry

In [33]:
cc=county_gdf.geometry

In [34]:
cc

0       POLYGON ((-85.38872 33.91304, -85.38088 33.873...
1       POLYGON ((-86.03044 31.61894, -86.00408 31.619...
2       POLYGON ((-86.00928 33.10164, -86.00917 33.090...
3       POLYGON ((-86.34851 30.99434, -86.35023 30.994...
4       POLYGON ((-86.14699 31.68045, -86.14711 31.663...
                              ...                        
3216    POLYGON ((-66.90748 18.25314, -66.90739 18.253...
3217    POLYGON ((-66.37968 17.94398, -66.38029 17.943...
3218    MULTIPOLYGON (((-66.13957 18.46232, -66.13956 ...
3219    POLYGON ((-66.02917 18.37590, -66.02828 18.376...
3220    POLYGON ((-66.85229 17.95500, -66.85280 17.955...
Name: geometry, Length: 3221, dtype: geometry

In [71]:
cc[0].get_coordinates()

AttributeError: 'Polygon' object has no attribute 'get_coordinates'

In [37]:
a=cc.get_coordinates()
a

,x,y
0,-85.388717,33.913044
0,-85.380885,33.873508
0,-85.379455,33.866291
0,-85.377426,33.856047
0,-85.376403,33.850656
...,...,...
3220,-66.833718,17.989763
3220,-66.835282,17.988274
3220,-66.835429,17.986323
3220,-66.836682,17.965971
